# Import data

Import data from file. Checks out the top 10 incorrect most popular submissions  and stores 
the predicates they belong to. Then, it creates a new dataframe with the top 3 most popular submissions for each one of these predicates.

In [ ]:
import pandas as pd 
# setting the display options
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

# Get the predicates with the most popular incorrect answers 
challenge = "dkZH6HJNQNLLDX6Aj"
data = pd.read_csv('popularity.csv', delimiter=';').sort_values(by='Popularity', ascending=False)
predicates = data.head(10)["Predicate"].unique().tolist()

df = pd.DataFrame()
for predicate in predicates:
    top = data[data["Predicate"] == predicate].head(3)
    df = pd.concat([df, top], ignore_index=True)

df.drop(columns=['Popularity'], inplace=True)
df

# Setup Databases

Setup databases for SpecAssistant and HiGenA.

In [58]:
import requests

def send_http_request(url: str, body=None, method="POST"):
    try:
        response = requests.request(method, url, json=body)
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            content_type = response.headers.get('Content-Type')
            if content_type and 'application/json' in content_type:
                return response.json()
            else:
                return response.text
        else:
            print(f"Request failed with status code {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
    return None


In [12]:
# Setup HiGenA
send_http_request(url="http://localhost:8080/hint/higena-setup",  body=[challenge])

An error occurred: Expecting value: line 1 column 1 (char 0)
None


In [22]:
# Setup SpecAssistant
send_http_request(url="http://localhost:8080/hint/specassistant-setup", body=[challenge], method="GET")

An error occurred: Expecting value: line 1 column 1 (char 0)


# Request hints

WARNING: Run only after setup is complete.

In [55]:
def gen_body_request(model, challenge, predicate, expression, hintGenType):
    obj = {
        "model": model + " pred " + predicate + " { " + expression + " }",
        "challenge": challenge,
        "predicate": predicate,
        "hintGenType": hintGenType
    }

    return obj

def get_hint(model, url, challenge, predicate, expression, hintGenType = "TED"):
    body = gen_body_request(model, challenge, predicate, expression, hintGenType)
    response = send_http_request(url, body)

    if response is not None:
        return pd.Series([response["hint"], response["nextExpr"], response["targetExpr"]])
    else:
        return pd.Series(["", "", ""])

## HiGenA
### TED policy
Generates hints using HiGenA with the path with the lowest TED.

In [59]:
model = "sig User {follows : set User,sees : set Photo,posts : set Photo,suggested : set User} sig Influencer extends User {} sig Photo {date : one Day} sig Ad extends Photo {} sig Day {}"
url = "http://localhost:8080/hint/higena-hint"

# Hint using Higena with TED
higenaTED = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
higenaTED[["hint", "next", "solution"]] = columns

higenaTED

,Challenge,Predicate,Expression,hint,next,solution
0,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the unique quantifier ('one') expression.,all i : Influencer | all u : User - i |i in u.follows,all i : Influencer | all u : User - i |i in u.follows
1,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows","Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.","all x : User, i : Influencer | x != i => i in x.follows","all x : User, i : Influencer | x != i => i in x.follows"
2,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the unique quantifier ('one') expression.,all x : Influencer | all y : User-x | y in follows.x,all x : Influencer | all y : User-x | y in follows.x
3,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all p : Photo | one u : User | u->p in posts,all p : Photo | one u : User | u->p in posts
4,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts,"Keep going! Instead of using inclusion operator ('in') to specify that some element(s) belong to a set, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all x : Photo | one posts.x,all x : Photo | one posts.x
5,dkZH6HJNQNLLDX6Aj,inv1,all p:Photo | some u:User | p in u.posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all p: Photo | one u : User | p in u.posts,all p: Photo | one u : User | p in u.posts
6,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows,"Keep going! Instead of using conjunction operator ('and') to combine two boolean expressions, try using equal operator ('=') to specify that the left side is equal to the right side.",all u : User | u.suggested = (u.follows.follows - u.follows - u ),all u : User | u.suggested = (u.follows.follows - u.follows - u )
7,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested","Keep going! It seems like you have unnecessary elements in your expression. You can try simplifying your expression by deleting the disjoint operator ('disj'). If you want to keep it, try to fix your expression another way and reach a different solution!","all u1,u2 : User | u1 in u2.follows.follows and u1 not in u2.follows => u1 in u2.suggested","all u1,u2:User | (u1!=u2 and u1 in u2.follows.follows and u1 not in u2.follows) <=> (u1 in u2.suggested)"
8,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows",Keep going! Consider adding a not equal operator ('!=') to specify that the left side is not equal to the right side. Think about how you can incorporate this within the conjunction operator ('and') expression.,"all u1,u2: User | u2 in u1.suggested implies u2 not in u1.follows and u2 in u1.follows.follows and u1 != u2","all u1,u2 : User | u2 in u1.suggested <=> u2 not in u1.follows and u2!=u1 and u2 in u1.follows.follows"
9,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad,"One step away from the solution! Instead of using existential quantifi

### Node Popularity policy

Generates hints using HiGenA with the path with the most popular submissions.

In [60]:
# Hint using Higena with Most popular submissions
higenaNode = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"], hintGenType="NODE_POISSON"), axis=1)
higenaNode[["hint", "next", "solution"]] = columns

higenaNode

,Challenge,Predicate,Expression,hint,next,solution
0,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,"Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.","all i : Influencer, u : User | u!=i => i in u.follows","all i : Influencer, u : User | u!=i => i in u.follows"
1,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows","Keep going! Instead of using signature of type User, try using signature of type Influencer to help satisfy the required property.","all i : Influencer, u : User | u!=i => i in u.follows","all i : Influencer, u : User | u!=i => i in u.follows"
2,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,"Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.","all i : Influencer, u : User | u!=i => i in u.follows","all i : Influencer, u : User | u!=i => i in u.follows"
3,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"Keep going! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all x : Photo | one posts.x,all x : Photo | one posts.x
4,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts,"Keep going! Instead of using inclusion operator ('in') to specify that some element(s) belong to a set, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all x : Photo | one posts.x,all x : Photo | one posts.x
5,dkZH6HJNQNLLDX6Aj,inv1,all p:Photo | some u:User | p in u.posts,"Keep going! Instead of using inclusion operator ('in') to specify that some element(s) belong to a set, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all x : Photo | one posts.x,all x : Photo | one posts.x
6,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows,"Keep going! Instead of using conjunction operator ('and') to combine two boolean expressions, try using equal operator ('=') to specify that the left side is equal to the right side.",all u : User | u.suggested = (u.follows.follows - u.follows - u ),all u : User | u.suggested = (u.follows.follows - u.follows - u )
7,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested","Keep going! Instead of using implication operator ('=>') to specify that if the left side is true, then the right side must also be true, try using equal operator ('=') to specify that the left side is equal to the right side.",all u : User | u.suggested = (u.follows.follows - u.follows - u ),all u : User | u.suggested = (u.follows.follows - u.follows - u )
8,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows","Keep going! Instead of using implication operator ('=>') to specify that if the left side is true, then the right side must also be true, try using equal operator ('=') to specify that the left side is equal to the right side.",all u : User | u.suggested = (u.follows.follows - u.follows - u ),all u : User | u.suggested = (u.follows.follows - u.follows - u )
9,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using universal quantifier ('all') to specify that all elements in a set satisfy a condition.",all u: User | all ad : Ad | ad in u.posts implies u.posts in Ad,all u: User | all ad : Ad | ad in u.post

### Edge Popularity policy
Generates hints using HiGenA with the path with the most popular transitions.

In [61]:
# Hint using Higena with Most popular submissions
higenaEdge = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"], hintGenType="REL_POISSON"), axis=1)
higenaEdge[["hint", "next", "solution"]] = columns

higenaEdge

,Challenge,Predicate,Expression,hint,next,solution
0,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the unique quantifier ('one') expression.,all i : Influencer | all u : User - i |i in u.follows,all i : Influencer | all u : User - i |i in u.follows
1,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows","Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.","all x : User, i : Influencer | x != i => i in x.follows","all x : User, i : Influencer | x != i => i in x.follows"
2,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the unique quantifier ('one') expression.,all x : Influencer | all y : User-x | y in follows.x,all x : Influencer | all y : User-x | y in follows.x
3,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"Keep going! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all x : Photo | one posts.x,all x : Photo | one posts.x
4,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts,Keep going! Consider adding a unique quantifier ('one') to specify that there is exactly one element in a set. Think about how you can incorporate this within the universal quantifier ('all') expression.,all p: Photo | one u : User | p in u.posts,all p: Photo | one u : User | p in u.posts
5,dkZH6HJNQNLLDX6Aj,inv1,all p:Photo | some u:User | p in u.posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all p: Photo | one u : User | p in u.posts,all p: Photo | one u : User | p in u.posts
6,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows,"Keep going! Instead of using conjunction operator ('and') to combine two boolean expressions, try using equal operator ('=') to specify that the left side is equal to the right side.",all u : User | u.suggested = (u.follows.follows - (u.follows + u)),all u : User | u.suggested = (u.follows.follows - (u.follows + u))
7,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested","Keep going! Instead of using implication operator ('=>') to specify that if the left side is true, then the right side must also be true, try using equal operator ('=') to specify that the left side is equal to the right side.",all u : User | u.suggested = (u.follows.follows - (u.follows + u)),all u : User | u.suggested = (u.follows.follows - (u.follows + u))
8,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows",Keep going! Consider adding a not equal operator ('!=') to specify that the left side is not equal to the right side. Think about how you can incorporate this within the conjunction operator ('and') expression.,"all u1,u2: User | u2 in u1.suggested implies u2 not in u1.follows and u2 in u1.follows.follows and u1 != u2","all u1,u2 : User | u2 in u1.suggested <=> u2 not in u1.follows and u2!=u1 and u2 in u1.follows.follows"
9,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using universal quantifier ('all')

## Spec Assistant
### Default policy
Generates hints using SpecAssistant default parameters.

In [62]:
# Compute policy
url = "http://localhost:8080/hint/compute-all-policies-for-rule?rule=TEDCOMPXxArrival"
send_http_request(url, [challenge], "POST")

''

In [63]:
# Hint using Spec Assistant default parameters
url = "http://localhost:8080/hint/spec-hint"
spec = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
spec[["hint", "next", "solution"]] = columns
spec

,Challenge,Predicate,Expression,hint,next,solution
0,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the one of expression.,(all ref0:(one Influencer)|(all ref1:(one (User - ref0))|(ref0 in (ref1 . (User <: follows))))),
1,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows",Near a solution! Consider adding a union operator ('+') to combine two sets. Think about how you can incorporate this within the one of expression.,(all ref0:(one (Influencer + User))|(all ref1:(one Influencer)|(ref1 in (ref0 . (User <: follows))))),
2,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the one of expression.,(all ref0:(one Influencer)|(all ref1:(one (User - ref0))|(ref1 in ((User <: follows) . ref0)))),
3,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",(all ref0:(one Photo)|(one ref1:(one User)|((ref1 -> ref0) in (User <: posts)))),
4,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts,"Near a solution! Instead of using signature of type Photo, try using signature of type User to help satisfy the required property.",(all ref0:(one User)|(ref0 in (User . (User <: posts)))),
5,dkZH6HJNQNLLDX6Aj,inv1,all p:Photo | some u:User | p in u.posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",(all ref0:(one Photo)|(one ref1:(one User)|(ref0 in (ref1 . (User <: posts))))),
6,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows,"One step away from the solution! Instead of using conjunction operator ('and') to combine two boolean expressions, try using disjunction operator ('or') to combine two boolean expressions.",(all ref0:(one User)|(((ref0 . (User <: suggested)) !in (ref0 . (User <: follows))) || ((ref0 . (User <: suggested)) in ((ref0 . (User <: follows)) . (User <: follows))))),
7,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested","One step away from the solution! Instead of using variable of type User, try using variable of type User to help satisfy the required property.","(all ref0:(one User)|(all ref1:(one User)|(disj[ref0,ref1] && (((ref0 !in (ref1 . (User <: follows))) && (ref0 in ((ref1 . (User <: follows)) . (User <: follows)))) => (ref0 in (ref0 . (User <: suggested)))))))",
8,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows","Keep going! Instead of using dot join operator ('.') to perform a relational join between sets or relations, try using difference operator ('-') to remove elements from a set.",(all ref0:(one User)|(all ref1:(one User)|((ref1 in (ref0 . (User <: suggested))) => ((ref1 !in (ref0 . (User <: follows))) && (ref1 in ((ref0 . (^ (User <: follows))) - ref0)))))),
9,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using universal quantifier ('all') to specify that all elements in a set satisfy a condition.",(all ref0:(one User)|(all ref1:(one Ad)|((ref1 in (ref0 . (User <: posts))) => ((ref0 . (User <: posts)) in Ad)))),


### TED policy

In [64]:
# Compute policy
url = "http://localhost:8080/hint/compute-all-policies-for-rule?rule=TED"
send_http_request(url)

An error occurred: Expecting value: line 1 column 1 (char 0)


In [65]:
# Generate hint
url = "http://localhost:8080/hint/spec-hint"
specTed = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
specTed[["hint", "next", "solution"]] = columns
specTed

,Challenge,Predicate,Expression,hint,next,solution
0,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the one of expression.,(all ref0:(one Influencer)|(all ref1:(one (User - ref0))|(ref0 in (ref1 . (User <: follows))))),
1,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows","Keep going! Instead of using signature of type User, try using signature of type Influencer to help satisfy the required property.",(all ref0:(one Influencer)|(all ref1:(one (User - ref0))|(ref0 in (ref1 . (User <: follows))))),
2,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the one of expression.,(all ref0:(one Influencer)|(all ref1:(one (User - ref0))|(ref1 in ((User <: follows) . ref0)))),
3,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",(all ref0:(one Photo)|(one ref1:(one User)|((ref1 -> ref0) in (User <: posts)))),
4,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts,Keep going! Consider adding a unique quantifier ('one') to specify that there is exactly one element in a set. Think about how you can incorporate this within the universal quantifier ('all') expression.,(all ref0:(one Photo)|(one ref1:(one User)|(ref0 in (ref1 . (User <: posts))))),
5,dkZH6HJNQNLLDX6Aj,inv1,all p:Photo | some u:User | p in u.posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",(all ref0:(one Photo)|(one ref1:(one User)|(ref0 in (ref1 . (User <: posts))))),
6,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows,"One step away from the solution! Instead of using conjunction operator ('and') to combine two boolean expressions, try using disjunction operator ('or') to combine two boolean expressions.",(all ref0:(one User)|(((ref0 . (User <: suggested)) !in (ref0 . (User <: follows))) || ((ref0 . (User <: suggested)) in ((ref0 . (User <: follows)) . (User <: follows))))),
7,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested",Keep going! It seems like the implication operator ('=>') is not in the right place. Try moving it to the inside of the universal quantifier ('all') expression. Try moving it so that you correctly ensure the required property.,(all ref0:(one User)|(all ref1:(one User)|(((ref0 !in (ref1 . (User <: follows))) && (ref0 in ((ref1 . (User <: follows)) . (User <: follows)))) => (ref0 in (ref1 . (User <: suggested)))))),
8,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows","Keep going! Instead of using implication operator ('=>') to specify that if the left side is true, then the right side must also be true, try using equivalence operator ('<=>') to specify the equivalence of the right and left side of the expression.",(all ref0:(one User)|(all ref1:(one User)|(((ref1 !in (ref0 . (User <: follows))) && (ref1 in ((ref0 . (User <: follows)) . (User <: follows)))) <=> (ref1 in (ref0 . (User <: suggested)))))),
9,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using universal quantifier ('all') to specify that all el

### Node Popularity policy

In [66]:
# Compute policy
url = "http://localhost:8080/hint/compute-all-policies-for-rule?rule=MAXFREQ"
send_http_request(url)

An error occurred: Expecting value: line 1 column 1 (char 0)


In [67]:
# Generate hint
url = "http://localhost:8080/hint/spec-hint"
specPopularNode = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
specPopularNode[["hint", "next", "solution"]] = columns
specPopularNode

,Challenge,Predicate,Expression,hint,next,solution
0,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,Keep going! Consider adding a disjunction operator ('or') to combine two boolean expressions. Think about how you can incorporate this within the universal quantifier ('all') expression.,(all ref0:(one Influencer)|(all ref1:(one User)|((ref0 = ref1) || (ref0 in (ref1 . (User <: follows)))))),
1,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows","Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.",(all ref0:(one User)|(all ref1:(one Influencer)|((ref1 !in ref0) => (ref1 in (ref0 . (User <: follows)))))),
2,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,Keep going! Consider adding a disjunction operator ('or') to combine two boolean expressions. Think about how you can incorporate this within the universal quantifier ('all') expression.,(all ref0:(one Influencer)|(all ref1:(one User)|((ref0 = ref1) || (ref0 in (ref1 . (User <: follows)))))),
3,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"Keep going! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",(all ref0:(one Photo)|(one ((User <: posts) . ref0))),
4,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts,Keep going! Consider adding a unique quantifier ('one') to specify that there is exactly one element in a set. Think about how you can incorporate this within the universal quantifier ('all') expression.,(all ref0:(one Photo)|(one ref1:(one User)|(ref0 in (ref1 . (User <: posts))))),
5,dkZH6HJNQNLLDX6Aj,inv1,all p:Photo | some u:User | p in u.posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",(all ref0:(one Photo)|(one ref1:(one User)|(ref0 in (ref1 . (User <: posts))))),
6,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows,"Keep going! Instead of using conjunction operator ('and') to combine two boolean expressions, try using equal operator ('=') to specify that the left side is equal to the right side.",(all ref0:(one User)|(((((ref0 . (User <: follows)) . (User <: follows)) - (ref0 . (User <: follows))) - ref0) = (ref0 . (User <: suggested)))),
7,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested","Keep going! Instead of using implication operator ('=>') to specify that if the left side is true, then the right side must also be true, try using equal operator ('=') to specify that the left side is equal to the right side.",(all ref0:(one User)|(((((ref0 . (User <: follows)) . (User <: follows)) - (ref0 . (User <: follows))) - ref0) = (ref0 . (User <: suggested)))),
8,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows","Keep going! Instead of using implication operator ('=>') to specify that if the left side is true, then the right side must also be true, try using equal operator ('=') to specify that the left side is equal to the right side.",(all ref0:(one User)|((((ref0 . (User <: follows)) . (User <: follows)) - (ref0 + (ref0 . (User <: follows)))) = (ref0 . (User <: suggested)))),
9,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using universal quantifier ('all') to specify that all elements in a set

### Edge Popularity policy

In [68]:
# Compute policy
url = "http://localhost:8080/hint/compute-all-policies-for-rule?rule=POPULARITY"
send_http_request(url)

An error occurred: Expecting value: line 1 column 1 (char 0)


In [69]:
# Generate hint
url = "http://localhost:8080/hint/spec-hint"
specPopularEdge = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
specPopularEdge[["hint", "next", "solution"]] = columns
specPopularEdge

,Challenge,Predicate,Expression,hint,next,solution
0,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,Keep going! It seems like the inclusion operator ('in') is not in the right place. Try moving it to the inside of the universal quantifier ('all') expression. Try moving it so that you correctly ensure the required property.,(all ref0:(one Influencer)|(ref0 in (User . (User <: follows)))),
1,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows","Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.",(all ref0:(one User)|(all ref1:(one Influencer)|((ref0 != ref1) => (ref1 in (ref0 . (User <: follows)))))),
2,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,Keep going! It seems like the universal quantifier ('all') is not in the right place. Try moving it so that you correctly ensure the required property.,(all ref0:(one User)|(ref0 in ((User <: follows) . Influencer))),
3,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"Keep going! Instead of using inclusion operator ('in') to specify that some element(s) belong to a set, try using dot join operator ('.') to perform a relational join between sets or relations.",(all ref0:(one Photo)|(some ((User <: posts) . ref0))),
4,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts,Keep going! Consider adding a conjunction operator ('and') to combine two boolean expressions. Think about how you can incorporate this within your expression to ensure the required property.,((all ref0:(one Photo)|(ref0 in (User . (User <: posts)))) && (all ref1:(one Photo)|(one ref2:(one User)|(ref1 in (ref2 . (User <: posts)))))),
5,dkZH6HJNQNLLDX6Aj,inv1,all p:Photo | some u:User | p in u.posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",(all ref0:(one Photo)|(one ref1:(one User)|(ref0 in (ref1 . (User <: posts))))),
6,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows,"Keep going! Instead of using conjunction operator ('and') to combine two boolean expressions, try using inclusion operator ('in') to specify that some element(s) belong to a set.",(all ref0:(one User)|((ref0 . (User <: suggested)) in (((ref0 . (User <: follows)) . (User <: follows)) - (ref0 . (User <: follows))))),
7,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested",Keep going! It seems like the conjunction operator ('and') is not in the right place. Try moving it to the inside of the universal quantifier ('all') expression. Try moving it so that you correctly ensure the required property.,(all ref0:(one User)|(((ref0 . (User <: suggested)) !in (ref0 . (User <: follows))) && ((ref0 . (User <: suggested)) in ((ref0 . (User <: follows)) . (User <: follows))))),
8,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows",Near a solution! Consider adding a not equal operator ('!=') to specify that the left side is not equal to the right side. Think about how you can incorporate this within the conjunction operator ('and') expression.,(all ref0:(one User)|(all ref1:(one User)|((ref1 in (ref0 . (User <: suggested))) => ((ref0 != ref1) && (ref1 !in (ref0 . (User <: follows))) && (ref1 in ((ref0 . (User <: follows)) . (User <: follows))))))),
9,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using universal quantifier ('all') to specify that a

# Export

Export hints to file.

In [70]:
df["HiGenATED"] = higenaTED["hint"]
df["HiGenAPopularNode"] = higenaNode["hint"]
df["HiGenAPopularEdge"] = higenaEdge["hint"]
df["specTed"] = specTed["hint"]
df["specPopularNode"] = specPopularNode["hint"]
df["specPopularEdge"] = specPopularEdge["hint"]
# Export hints csv
df.to_csv("hints.csv", index=False, sep=";")


In [71]:
df

,Challenge,Predicate,Expression,HiGenATED,HiGenAPopularNode,HiGenAPopularEdge,specTed,specPopularNode,specPopularEdge
0,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the unique quantifier ('one') expression.,"Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.",Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the unique quantifier ('one') expression.,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the one of expression.,Keep going! Consider adding a disjunction operator ('or') to combine two boolean expressions. Think about how you can incorporate this within the universal quantifier ('all') expression.,Keep going! It seems like the inclusion operator ('in') is not in the right place. Try moving it to the inside of the universal quantifier ('all') expression. Try moving it so that you correctly ensure the required property.
1,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows","Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.","Keep going! Instead of using signature of type User, try using signature of type Influencer to help satisfy the required property.","Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.","Keep going! Instead of using signature of type User, try using signature of type Influencer to help satisfy the required property.","Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.","Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression."
2,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the unique quantifier ('one') expression.,"Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.",Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the unique quantifier ('one') expression.,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the one of expression.,Keep going! Consider adding a disjunction operator ('or') to combine two boolean expressions. Think about how you can incorporate this within the universal quantifier ('all') expression.,Keep going! It seems like the universal quantifier ('all') is not in the right place. Try moving it so that you correctly ensure the required property.
3,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements